# References
* This notebook shows how to use the `classifierclass.py` file to trian the spike features extracted from `network.py`.
* [Neural Nets](http://neuralnetworksanddeeplearning.com/chap3.html)
* [Randombackprop](https://github.com/xuexue/randombp/blob/master/randombp.py)
* [Randombackprop](https://github.com/sangyi92/feedback_alignment/blob/master/RFA.ipynb)
* [Backprop](http://blog.aloni.org/posts/backprop-with-tensorflow/)
* [Initializers](https://towardsdatascience.com/hyper-parameters-in-action-part-ii-weight-initializers-35aee1a28404)
* [Dropout](https://github.com/pinae/TensorFlow-MNIST-example/blob/master/fully-connected.py)
* [Softmax](https://stackoverflow.com/questions/34240703/what-is-logits-softmax-and-softmax-cross-entropy-with-logits)
* [SoftmaxLogits](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)

# Imports

In [1]:
import numpy as np
import h5py, sys, os, time, pickle
GPU =      True
sys.path.insert(0, "/home/ruthvik/Desktop/Summer 2017/SpykeFlow")
USE_FASTER_METHOD =   True
if(not GPU):
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import numpy as np
import spykeflow as sf
import matplotlib.pyplot as plt
from keras import backend
from spykeflow import nn_classifierclass as cls

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Load Data

In [2]:
filename = '../../spiking_networks/emnist_train_pool1_acc_cont_spike_features_inh_False_conv1maps_30.h5'
with h5py.File(filename, 'r') as hf:
    emnist_train_images = hf['pool1_spike_features'][:].astype(np.float32)

#emnist_train_images[np.where(emnist_train_images>=1)] = 1
print('Total train features:{}'.format(emnist_train_images.shape[0]))


filename = '../../spiking_networks/emnist_test_pool1_acc_cont_spike_features_inh_False_conv1maps_30.h5'
with h5py.File(filename, 'r') as hf:
    emnist_test_images = hf['pool1_spike_features'][:].astype(np.float32)
#emnist_test_images[np.where(emnist_test_images>=1)] = 1
print('test features:{}'.format(emnist_test_images.shape[0]))


filehandle = open('../../spiking_networks/emnist_train_y.pkl','rb')
emnist_train_labels = pickle.load(filehandle).astype(np.int).tolist()
filehandle.close()
#emnist_train_labels = to_categorical(emnist_train_labels, num_classes=47)


filehandle = open('../../spiking_networks/emnist_test_y.pkl','rb')
emnist_test_labels = pickle.load(filehandle).astype(np.int)
filehandle.close()
#emnist_test_labels = to_categorical(emnist_test_labels, num_classes=47)
print('Length of test labels:{}'.format(emnist_test_labels.shape[0]))

Total train features:112799
test features:18800
Length of test labels:18800


# Setup a NN classifier and train with spike counts (integer spike feature)

In [3]:
n_classes = 47
n_hidden = 1
network_structure = [emnist_train_images.shape[1],1500,n_classes]
#network_structure = [train_pool1_spike_features.shape[1],n_classes]
activation_fns = ['sigmoid']*(n_hidden)+['softmax']
#activation_fns = ['tanh']*(n_hidden)+['softmax']
#activation_fns = ['relu']*(n_hidden)+['softmax']
#activation_fns = ['swish']*(n_hidden)+['softmax']
#activation_fns = ['softmax']
#sys.exit()
#weight_init = 'he_uniform'
weight_init = 'glorot_uniform'
eta_drop_type = 'plateau'
lmbda = 0.000
batch_size = 5
eta = 0.001

log_path = '/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/emnist/emnist_regular_backprop_final'+''.\
join(activation_fns+['-',weight_init,'-',eta_drop_type,str(-lmbda)])+'/eta'+str(-eta)
print(log_path)
repeats = 10
all_histories = []
for repeat in range(repeats):
    print('Repeat:{}'.format(repeat))
    backend.clear_session()
    neural_net = cls.Classifier(train_data=(emnist_train_images,emnist_train_labels),
                                test_data=(emnist_test_images,emnist_test_labels),
                                network_structure=network_structure,activation_fns=activation_fns,
                                epochs=15,eta=eta,lmbda=lmbda,verbose=1,plots=False,optimizer='adam',
                                eta_decay_factor=1.007,patience=8,eta_drop_type=eta_drop_type,
                                epochs_drop=1, val_frac=0.09,drop_out=0.0,ip_lyr_drop_out=0.0,
                                leaky_alpha=0.1,leaky_relu=False,weight_init=weight_init,
                                bias_init=0.1,batch_size=batch_size,log_path=log_path)
    neural_net.keras_fcn_classifier()
    all_histories.append(neural_net.history)

/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/emnist/emnist_regular_backprop_finalsigmoidsoftmax-glorot_uniform-plateau-0.0/eta-0.001




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              5446500   
_________________________________________________________________
activation_1 (Activation)    (None, 1500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 47)                70547     
_________________________________________________________________
activation_2 (Activation)    (None, 47)                0         
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________
None

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 102647 sampl

102647/102647 [==============================] - 154s 1ms/step - loss: 0.0650 - acc: 0.9760 - val_loss: 0.5693 - val_acc: 0.8560

Epoch 00005: val_acc did not improve from 0.85796
Epoch 6/15
102647/102647 [==============================] - 151s 1ms/step - loss: 0.0418 - acc: 0.9850 - val_loss: 0.5809 - val_acc: 0.8592

Epoch 00006: val_acc improved from 0.85796 to 0.85924, saving model to weights.best.hdf5
Epoch 7/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.0266 - acc: 0.9911 - val_loss: 0.6222 - val_acc: 0.8595

Epoch 00007: val_acc improved from 0.85924 to 0.85954, saving model to weights.best.hdf5
Epoch 8/15
102647/102647 [==============================] - 151s 1ms/step - loss: 0.0172 - acc: 0.9952 - val_loss: 0.6480 - val_acc: 0.8599

Epoch 00008: val_acc improved from 0.85954 to 0.85993, saving model to weights.best.hdf5
Epoch 9/15
102647/102647 [==============================] - 151s 1ms/step - loss: 0.0107 - acc: 0.9976 - val_loss: 0.6528 - val_ac


Epoch 00003: val_acc improved from 0.85185 to 0.85628, saving model to weights.best.hdf5
Epoch 4/15
102647/102647 [==============================] - 154s 1ms/step - loss: 0.1068 - acc: 0.9600 - val_loss: 0.5340 - val_acc: 0.8577

Epoch 00004: val_acc improved from 0.85628 to 0.85766, saving model to weights.best.hdf5
Epoch 5/15
102647/102647 [==============================] - 154s 1ms/step - loss: 0.0658 - acc: 0.9755 - val_loss: 0.5652 - val_acc: 0.8576

Epoch 00005: val_acc did not improve from 0.85766
Epoch 6/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.0416 - acc: 0.9851 - val_loss: 0.6012 - val_acc: 0.8558

Epoch 00006: val_acc did not improve from 0.85766
Epoch 7/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.0273 - acc: 0.9905 - val_loss: 0.6248 - val_acc: 0.8589

Epoch 00007: val_acc improved from 0.85766 to 0.85894, saving model to weights.best.hdf5
Epoch 8/15
102647/102647 [==============================] - 153s 1ms

Epoch 2/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.2980 - acc: 0.8980 - val_loss: 0.4724 - val_acc: 0.8555

Epoch 00002: val_acc improved from 0.84712 to 0.85550, saving model to weights.best.hdf5
Epoch 3/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.1778 - acc: 0.9357 - val_loss: 0.5051 - val_acc: 0.8534

Epoch 00003: val_acc did not improve from 0.85550
Epoch 4/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.1064 - acc: 0.9614 - val_loss: 0.5329 - val_acc: 0.8587

Epoch 00004: val_acc improved from 0.85550 to 0.85875, saving model to weights.best.hdf5
Epoch 5/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.0650 - acc: 0.9762 - val_loss: 0.5600 - val_acc: 0.8585

Epoch 00005: val_acc did not improve from 0.85875
Epoch 6/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.0430 - acc: 0.9848 - val_loss: 0.5924 - val_acc: 0.8584

Epoch 00006: val_

Train on 102647 samples, validate on 10152 samples
Epoch 1/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.6124 - acc: 0.8087 - val_loss: 0.4688 - val_acc: 0.8456

Epoch 00001: val_acc improved from -inf to 0.84565, saving model to weights.best.hdf5
Epoch 2/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.2975 - acc: 0.8979 - val_loss: 0.4628 - val_acc: 0.8532

Epoch 00002: val_acc improved from 0.84565 to 0.85323, saving model to weights.best.hdf5
Epoch 3/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.1785 - acc: 0.9350 - val_loss: 0.5028 - val_acc: 0.8554

Epoch 00003: val_acc improved from 0.85323 to 0.85540, saving model to weights.best.hdf5
Epoch 4/15
102647/102647 [==============================] - 152s 1ms/step - loss: 0.1067 - acc: 0.9615 - val_loss: 0.5330 - val_acc: 0.8565

Epoch 00004: val_acc improved from 0.85540 to 0.85648, saving model to weights.best.hdf5
Epoch 5/15
102647/102647 [======

Train on 102647 samples, validate on 10152 samples
Epoch 1/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.6110 - acc: 0.8092 - val_loss: 0.4554 - val_acc: 0.8536

Epoch 00001: val_acc improved from -inf to 0.85362, saving model to weights.best.hdf5
Epoch 2/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.2978 - acc: 0.8972 - val_loss: 0.4629 - val_acc: 0.8562

Epoch 00002: val_acc improved from 0.85362 to 0.85619, saving model to weights.best.hdf5
Epoch 3/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.1780 - acc: 0.9355 - val_loss: 0.5076 - val_acc: 0.8517

Epoch 00003: val_acc did not improve from 0.85619
Epoch 4/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.1061 - acc: 0.9608 - val_loss: 0.5357 - val_acc: 0.8583

Epoch 00004: val_acc improved from 0.85619 to 0.85825, saving model to weights.best.hdf5
Epoch 5/15
102647/102647 [==============================] - 153s 1ms/st

In [5]:
best_accuracies = [item['best_test_acc'] for item in all_histories]
#best_accuracies = [86.04, 86.19, 85.8, 86.01, 85.81]
last_accuracies = [item['last_test_acc'] for item in all_histories]
#last_accuracies = [86.07, 86.24, 85.88, 86.01, 85.84]
best_accuracies = np.array(best_accuracies)*100
print('best test:{}, mean test:{}, std test:{}'.format(best_accuracies.max(),
                                                                        best_accuracies.mean(),
                                                                         best_accuracies.std()))

last_accuracies = np.array(last_accuracies)*100
print('best final test:{}, mean final test:{}, std final test:{}'.format(last_accuracies.max(),
                                                                        last_accuracies.mean(),
                                                                         last_accuracies.std()))

best test:85.9893617021, mean test:85.8356382979, std test:0.128134091502
best final test:86.085106383, mean final test:85.8388297872, std final test:0.124836923955


# Save the results to pkl

In [10]:
picklefile = '../outputs/journal_resubmit/spike_count_recording.pkl' 
output1 = open(picklefile,'wb')
pickle.dump(all_histories,output1)
output1.close()
print('pickle file written to:{}'.format(picklefile))

pickle file written to:../outputs/journal_resubmit/spike_count_recording.pkl


## Verify the written data

In [12]:
picklefile = open('../outputs/journal_resubmit/spike_count_recording.pkl', 'rb') 
loaded_spike_count_record_all_histories = pickle.load(picklefile)
picklefile.close()
#loaded_spike_count_record_all_histories

# Setup a NN classifier and train with population activity (binary spike features)

In [6]:
emnist_train_images[np.where(emnist_train_images>=1)] = 1
emnist_test_images[np.where(emnist_test_images>=1)] = 1

In [7]:
n_classes = 47
n_hidden = 1
network_structure = [emnist_train_images.shape[1],1500,n_classes]
#network_structure = [train_pool1_spike_features.shape[1],n_classes]
activation_fns = ['sigmoid']*(n_hidden)+['softmax']
#activation_fns = ['tanh']*(n_hidden)+['softmax']
#activation_fns = ['relu']*(n_hidden)+['softmax']
#activation_fns = ['swish']*(n_hidden)+['softmax']
#activation_fns = ['softmax']
#sys.exit()
#weight_init = 'he_uniform'
weight_init = 'glorot_uniform'
eta_drop_type = 'plateau'
lmbda = 0.000
batch_size = 5
eta = 0.001

log_path = '/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/emnist/emnist_regular_backprop_final_binary'+''.\
join(activation_fns+['-',weight_init,'-',eta_drop_type,str(-lmbda)])+'/eta'+str(-eta)
print(log_path)
repeats = 10
pop_record_all_histories = []
for repeat in range(repeats):
    print('Repeat:{}'.format(repeat))
    backend.clear_session()
    neural_net = cls.Classifier(train_data=(emnist_train_images,emnist_train_labels),
                                test_data=(emnist_test_images,emnist_test_labels),
                                network_structure=network_structure,activation_fns=activation_fns,
                                epochs=15,eta=eta,lmbda=lmbda,verbose=1,plots=False,optimizer='adam',
                                eta_decay_factor=1.007,patience=8,eta_drop_type=eta_drop_type,
                                epochs_drop=1, val_frac=0.09,drop_out=0.0,ip_lyr_drop_out=0.0,
                                leaky_alpha=0.1,leaky_relu=False,weight_init=weight_init,
                                bias_init=0.1,batch_size=batch_size,log_path=log_path)
    neural_net.keras_fcn_classifier()
    pop_record_all_histories.append(neural_net.history)

/home/ruthvik/Desktop/Summer 2017/tf_graph_outputs/emnist/emnist_regular_backprop_final_binarysigmoidsoftmax-glorot_uniform-plateau-0.0/eta-0.001
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              5446500   
_________________________________________________________________
activation_1 (Activation)    (None, 1500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 47)                70547     
_________________________________________________________________
activation_2 (Activation)    (None, 47)                0         
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________
None
Train on 102647 samples, validate on 10152 samples
Epoch 1/15
102647/102647 [==============================] - 15

102647/102647 [==============================] - 152s 1ms/step - loss: 0.0037 - acc: 0.9998 - val_loss: 0.7271 - val_acc: 0.8571

Epoch 00015: val_acc did not improve from 0.85914
Testing model at the end
18800/18800 [==============================] - 1s 29us/step
acc: 85.78%
Testing the best model
18800/18800 [==============================] - 0s 26us/step
acc: 85.67%
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              5446500   
_________________________________________________________________
activation_1 (Activation)    (None, 1500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 47)                70547     
_________________________________________________________________
activation_2 (Activation)    (None, 47)                0         
Total params: 5,517,047
Trainable 

102647/102647 [==============================] - 154s 2ms/step - loss: 0.0046 - acc: 0.9997 - val_loss: 0.7146 - val_acc: 0.8587

Epoch 00013: val_acc did not improve from 0.86032
Epoch 14/15
102647/102647 [==============================] - 154s 2ms/step - loss: 0.0046 - acc: 0.9997 - val_loss: 0.7203 - val_acc: 0.8590

Epoch 00014: val_acc did not improve from 0.86032
Epoch 15/15
102647/102647 [==============================] - 155s 2ms/step - loss: 0.0045 - acc: 0.9997 - val_loss: 0.7242 - val_acc: 0.8588

Epoch 00015: val_acc did not improve from 0.86032
Testing model at the end
18800/18800 [==============================] - 1s 28us/step
acc: 85.70%
Testing the best model
18800/18800 [==============================] - 1s 27us/step
acc: 85.77%
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1500)              5446500   
______________________________________________


Epoch 00011: val_acc improved from 0.86013 to 0.86062, saving model to weights.best.hdf5
Epoch 12/15
102647/102647 [==============================] - 157s 2ms/step - loss: 0.0043 - acc: 0.9997 - val_loss: 0.7183 - val_acc: 0.8601

Epoch 00012: val_acc did not improve from 0.86062
Epoch 13/15
102647/102647 [==============================] - 155s 2ms/step - loss: 0.0039 - acc: 0.9998 - val_loss: 0.7232 - val_acc: 0.8599

Epoch 00013: val_acc did not improve from 0.86062
Epoch 14/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.0038 - acc: 0.9998 - val_loss: 0.7291 - val_acc: 0.8597

Epoch 00014: val_acc did not improve from 0.86062
Epoch 15/15
102647/102647 [==============================] - 153s 1ms/step - loss: 0.0037 - acc: 0.9998 - val_loss: 0.7343 - val_acc: 0.8602

Epoch 00015: val_acc did not improve from 0.86062
Testing model at the end
18800/18800 [==============================] - 0s 25us/step
acc: 85.87%
Testing the best model
18800/18800 [=========

Epoch 10/15
102647/102647 [==============================] - 162s 2ms/step - loss: 0.0078 - acc: 0.9986 - val_loss: 0.6789 - val_acc: 0.8595

Epoch 00010: val_acc did not improve from 0.85983
Epoch 11/15
102647/102647 [==============================] - 158s 2ms/step - loss: 0.0054 - acc: 0.9995 - val_loss: 0.6914 - val_acc: 0.8591

Epoch 00011: val_acc did not improve from 0.85983
Epoch 12/15
102647/102647 [==============================] - 160s 2ms/step - loss: 0.0044 - acc: 0.9997 - val_loss: 0.7054 - val_acc: 0.8603

Epoch 00012: val_acc improved from 0.85983 to 0.86032, saving model to weights.best.hdf5
Epoch 13/15
102647/102647 [==============================] - 161s 2ms/step - loss: 0.0042 - acc: 0.9998 - val_loss: 0.7127 - val_acc: 0.8600

Epoch 00013: val_acc did not improve from 0.86032
Epoch 14/15
102647/102647 [==============================] - 155s 2ms/step - loss: 0.0041 - acc: 0.9998 - val_loss: 0.7166 - val_acc: 0.8596

Epoch 00014: val_acc did not improve from 0.86032
E

102647/102647 [==============================] - 160s 2ms/step - loss: 0.0193 - acc: 0.9939 - val_loss: 0.6483 - val_acc: 0.8604

Epoch 00008: val_acc improved from 0.85973 to 0.86042, saving model to weights.best.hdf5
Epoch 9/15
102647/102647 [==============================] - 157s 2ms/step - loss: 0.0118 - acc: 0.9970 - val_loss: 0.6701 - val_acc: 0.8596

Epoch 00009: val_acc did not improve from 0.86042
Epoch 10/15
102647/102647 [==============================] - 157s 2ms/step - loss: 0.0077 - acc: 0.9987 - val_loss: 0.6803 - val_acc: 0.8596

Epoch 00010: val_acc did not improve from 0.86042
Epoch 11/15
102647/102647 [==============================] - 156s 2ms/step - loss: 0.0053 - acc: 0.9995 - val_loss: 0.6992 - val_acc: 0.8601

Epoch 00011: val_acc did not improve from 0.86042
Epoch 12/15
102647/102647 [==============================] - 159s 2ms/step - loss: 0.0043 - acc: 0.9997 - val_loss: 0.7052 - val_acc: 0.8590

Epoch 00012: val_acc did not improve from 0.86042
Epoch 13/15
10

In [8]:
best_accuracies = [item['best_test_acc'] for item in pop_record_all_histories]

last_accuracies = [item['last_test_acc'] for item in pop_record_all_histories]

best_accuracies = np.array(best_accuracies)*100
print('best test:{}, mean test:{}, std test:{}'.format(best_accuracies.max(),
                                                                        best_accuracies.mean(),
                                                                         best_accuracies.std()))

last_accuracies = np.array(last_accuracies)*100
print('best final test:{}, mean final test:{}, std final test:{}'.format(last_accuracies.max(),
                                                                        last_accuracies.mean(),
                                                                         last_accuracies.std()))

best test:85.920212766, mean test:85.8015957447, std test:0.117336374614
best final test:85.9574468085, mean final test:85.825, std final test:0.101829407135


## Save results to pkl file

In [13]:
picklefile = '../outputs/journal_resubmit/population_recording.pkl' 
output1 = open(picklefile,'wb')
pickle.dump(pop_record_all_histories,output1)
output1.close()
print('pickle file written to:{}'.format(picklefile))

pickle file written to:../outputs/journal_resubmit/population_recording.pkl


## Verify the written data

In [15]:
picklefile = open('../outputs/journal_resubmit/population_recording.pkl', 'rb') 
loaded_pop_record_all_histories = pickle.load(picklefile)
picklefile.close()
#loaded_pop_record_all_histories